In [1]:
# Import data
import numpy as np
import pandas as pd 
import os

In [2]:
# Import package
import matplotlib.pyplot as plt
import seaborn as sns
from pylab import rcParams
import matplotlib.cm as cm

import sklearn
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder               # conversion
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit

from sklearn.ensemble import RandomForestClassifier          # Random Forest
from sklearn.svm import SVC, LinearSVC                       # SVC
from sklearn.linear_model import LogisticRegression          # Logic Regression
from sklearn.neighbors import KNeighborsClassifier           # KNN
from sklearn.naive_bayes import GaussianNB                   # Naive Bayes
from sklearn.tree import DecisionTreeClassifier              # Decision Tree
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier     

from sklearn.metrics import classification_report, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.ensemble import VotingClassifier

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

import warnings
warnings.filterwarnings('ignore')

get_ipython().magic('matplotlib inline')


In [3]:
import os
print(os.getcwd())
# read data file
df=pd.read_csv("telcocustomerchurn.csv")
df.head(10)
df.shape
df.describe()

C:\Users\lenovo\ChurnPredict


,Unnamed: 0_demographics,Count_demographics,Age,Number of Dependents,Unnamed: 0_location,Count_location,Zip Code,Latitude,Longitude,Unnamed: 0,...,Unnamed: 0_status,Count_status,Satisfaction Score,Churn Value,Churn Score,CLTV,Unnamed: 0_customerc,LoyaltyID,Tenure,Monthly Charges
count,7043.000000,7043.0,7043.000000,7043.000000,7043.000000,7043.0,7043.000000,7043.000000,7043.000000,7043.000000,...,7043.000000,7043.0,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000
mean,3521.000000,1.0,46.509726,0.468692,3521.000000,1.0,93486.071134,36.197455,-119.756684,3521.000000,...,3521.000000,1.0,3.244924,0.265370,58.505040,4400.295755,3521.000000,550382.651001,32.371149,64.761692
std,2033.283305,0.0,16.750352,0.962802,2033.283305,0.0,1856.768045,2.468929,2.154425,2033.283305,...,2033.283305,0.0,1.201657,0.441561,21.170031,1183.057152,2033.283305,260776.118690,24.559481,30.090047
min,0.000000,1.0,19.000000,0.000000,0.000000,1.0,90001.000000,32.555828,-124.301372,0.000000,...,0.000000,1.0,1.000000,0.000000,5.000000,2003.000000,0.000000,100346.000000,0.000000,18.250000
25%,1760.500000,1.0,32.000000,0.000000,1760.500000,1.0,92101.000000,33.990646,-121.788090,1760.500000,...,1760.500000,1.0,3.000000,0.000000,40.000000,3469.000000,1760.500000,323604.500000,9.000000,35.500000
50%,3521.000000,1.0,46.000000,0.000000,3521.000000,1.0,93518.000000,36.205465,-119.595293,3521.000000,...,3521.000000,1.0,3.000000,0.000000,61.000000,4527.000000,3521.000000,548704.000000,29.000000,70.350000
75%,5281.500000,1.0,60.000000,0.000000,5281.500000,1.0,95329.000000,38.161321,-117.969795,5281.500000,...,5281.500000,1.0,4.000000,1.000000,75.500000,5380.500000,5281.500000,776869.000000,55.000000,89.850000
max,7042.000000,1.0,80.000000,9.000000,7042.000000,1.0,96150.000000,41.962127,-114.192901,7042.000000,...,7042.000000,1.0,5.000000,1.000000,96.000000,6500.000000,7042.000000,999912.000000,72.000000,118.750000


### **Meaning of All the Variables**
#### Churn Predict
- Loyalty ID:A unique identifier for each customer's loyalty program membership
- Customer ID:A unique identifier for each customer. 
- Senior Citizen:Indicates if the customer is a senior citizen (Yes/No).
- Partner: Indicates if the customer has a partner or spouse (Yes/No).
- Dependents: Indicates if the customer has family member (Yes/No).
- Tenure:The number of months the customer has stayed with the company.
- Phone Service: Indicates if the customer has phone service (Yes/No).
- Multiple Lines:Indicates if the customer has multiple phone lines (Yes/No).
- Internet Service: Type of internet service the customer has (DSL, Fiber optic, No)
- Online Security: Indicates if the customer has subscribed to online security services (Yes/No).
- Online Backup: Indicates if the customer has subscribed to online backup services (Yes/No). 
- Device Protection:  Indicates if the customer has subscribed to device protection services (Yes/No).
- Tech Support: Indicates if the customer has subscribed to technical support services (Yes/No).
- Streaming TV: Indicates if the customer has subscribed to streaming TV services (Yes/No).
- Streaming Movies: Indicates if the customer has subscribed to streaming movie services (Yes/No). 
- Contract: Type of contract the customer has (Month-to-month, One year, Two year).
- Paperless Billing: Indicates if the customer has opted for paperless billing (Yes/No).
- Payment Method: The payment method used by the customer (Bank transfer, Credit card, Electronic check, Mailed check). 
- Monthly Charges: The amount charged to the customer every month. 
- Total Charges: The total amount charged to the customer for the entire tenure
- Churn:  Indicates if the customer has discontinued the service (Yes/No). 

#### Demographics
- CustomerID: A unique ID that identifies each customer.
- Count: A value used in reporting/dashboarding to sum up the number of customers in a filtered set.~~(All 1, could be delete derictly)~~
- Gender: The customer’s gender (Male/Female)
- Age: The customer’s current age, in years, at the time the fiscal quarter ended.
- Senior Citizen: Indicates if the customer is 65 or older (Yes/No)
- Married: Indicates if the customer is married (Yes/No)
- Dependents: Indicates if the customer lives with any dependents: (Yes/No).
- Number of Dependents: Indicates the number of dependents that live with the customer.

#### Location:(?Precision)

- Location ID:A unique ID that identifies each location
- Customer ID: A unique ID that identifies each customer.
- Count: A value used in reporting/dashboarding to sum up the number of customers in a filtered set.
- Country: The country of the customer’s primary residence.~~(All US, could delete directly)~~
- State: The state of the customer’s primary residence.~~(All California, could be deleted directly)~~
- City: The city of the customer’s primary residence.
- Zip Code: The zip code of the customer’s primary residence.~~(strongly correlated to city?)~~
- Lat Long: The combined latitude and longitude of the customer’s primary residence.
- Latitude: The latitude of the customer’s primary residence.
- Longitude: The longitude of the customer’s primary residence.


#### Population: show the population of each zip code place
- ID: A unique ID that identifies each row.
- Zip Code: The zip code of the customer’s primary residence.
- Population: A current population estimate for the entire Zip Code area.

#### Services
- Service ID: A unique ID that identifies each types of service a customer subscribes to.
- CustomerID: A unique ID that identifies each customer.
- Count: A value used in reporting/dashboarding to sum up the number of customers in a filtered set.(All 1)
- Quarter: The fiscal quarter that the data has been derived from (All Q3).
- Referred a Friend: Indicates if the customer has ever referred a friend or family member to this company (Yes/No)
- Number of Referrals: Indicates the number of referrals to date that the customer has made.
- Tenure in Months: Indicates the total amount of months that the customer has been with the company by the end of the quarter specified above.
- Offer: Identifies the last marketing offer that the customer accepted, if applicable (Values include None/ Offer A/ Offer B/Offer C/Offer D/Offer E)
- Phone Service: Indicates if the customer subscribes to home phone service with the company (Yes/No)
- Avg Monthly Long Distance Charges: Indicates the customer’s average long distance charges, calculated to the end of the quarter specified above.
- Multiple Lines: Indicates if the customer subscribes to multiple telephone lines with the company(Yes/No)
- Internet Service: Indicates if the customer subscribes to Internet service with the company(Yes/No)
- Internet Type:Type of internet service the customer has(Fiber Optic/Cable/DSL/None)
- Avg Monthly GB Download: Indicates the customer’s average download volume in gigabytes, calculated to the end of the quarter specified above.
- Online Security: Indicates if the customer subscribes to an additional online security service provided by the company: (Yes/No)
- Online Backup: Indicates if the customer subscribes to an additional online backup service provided by the company: (Yes/No)
- Device Protection Plan: Indicates if the customer subscribes to an additional device protection plan for their Internet equipment provided by the company (Yes/No)
- Premium Tech Support: Indicates if the customer subscribes to an additional technical support plan from the company with reduced wait times (Yes/No)
- Streaming TV: Indicates if the customer uses their Internet service to stream television programing from a third party provider: (Yes/No)
- Streaming Movies: Indicates if the customer uses their Internet service to stream movies from a third party provider: (Yes/No)
- Streaming Music: Indicates if the customer uses their Internet service to stream music from a third party provider: (Yes/No) 
- Unlimited Data: Indicates if the customer has paid an additional monthly fee to have unlimited data downloads/uploads: (Yes/No)
- Contract: Indicates the customer’s current contract type: (Month-to-Month/One Year/Two Year)
- Paperless Billing: Indicates if the customer has chosen paperless billing: (Yes/No)
- Payment Method: Indicates how the customer pays their bill: (Bank Withdrawal/Credit Card/Mailed Check)
- Monthly Charge: Indicates the customer’s current total monthly charge for all their services from the company.
- Total Charges: Indicates the customer’s total charges, calculated to the end of the quarter specified above.
- Total Refunds: Indicates the customer’s total refunds, calculated to the end of the quarter specified above.
- Total Extra Data Charges: Indicates the customer’s total charges for extra data downloads above those specified in their plan, by the end of the quarter specified above.
- Total Long Distance Charges: Indicates the customer’s total charges for long distance above those specified in their plan, by the end of the quarter specified above.
- Total Revenue:Total Revenue= `Total Charges`- `Total Refunds`+`Total Extra Data Charges`+`Total Long Distance Charges`
#### Status
- Status ID:An unique ID that identifies each customer's statue
- CustomerID: A unique ID that identifies each customer.
- Count: A value used in reporting/dashboarding to sum up the number of customers in a filtered set.(ALL 1)
- Quarter: The fiscal quarter that the data has been derived from (ALL Q3).
- Satisfaction Score: A customer’s overall satisfaction rating of the company from 1 (Very Unsatisfied) to 5 (Very Satisfied).(1-5)
- Customer Status: Indicates the status of the customer at the end of the quarter: (Churned/Stayed/Joined)
- Churn Label: Yes = the customer left the company this quarter. No = the customer remained with the company. Directly related to Churn Value.(Yes/No)
- Churn Value: 1 = the customer left the company this quarter. 0 = the customer remained with the company. Directly related to Churn Label.(1/0)
- Churn Score: A value from 0-100 that is calculated using the predictive tool IBM SPSS Modeler. The model incorporates multiple factors known to cause churn. The higher the score, the more likely the customer will churn.(1-100)
- Churn Score Category: A calculation that assigns a Churn Score to one of the following categories: 0-10, 11-20, 21-30, 31-40, 41-50, 51-60, 61-70, 71-80, 81-90, and 91-100(Numerical to nominal,equal-interval binning)
- CLTV: Customer Lifetime Value. A predicted CLTV is calculated using corporate formulas and existing data. The higher the value, the more valuable the customer. High value customers should be monitored for churn.
- CLTV Category: A calculation that assigns a CLTV value to one of the following categories: 2000-2500, 2501-3000, 3001-3500, 3501-4000, 4001-4500, 4501-5000, 5001-5500, 5501-6000, 6001-6500, and 6501-7000.

- Churn Category: A high-level category for the customer’s reason for churning: Attitude, Competitor, Dissatisfaction, Other, Price. When they leave the company, all customers are asked about their reasons for leaving. Directly related to Churn Reason.（Attitude/Competitor/Dissatisfaction/Price/Other)
- Churn Reason: A customer’s specific reason for leaving the company. Directly related to Churn Category.

In [4]:

df['Total Charges_customerc'].replace('', np.nan, inplace=True)
df['Total Charges_customerc'] = pd.to_numeric(df['Total Charges_customerc'], errors='coerce')
df['Total Charges_customerc'] = df['Total Charges_customerc'].astype(float)



for col in df.select_dtypes(include=['object']).columns:
    if df[col].nunique() < 50:  
        df[col] = df[col].astype('category')

for col in df.columns:
    print(f"{col}: {df[col].dtype}")


Unnamed: 0_demographics: int64
Customer ID: object
Count_demographics: int64
Gender: category
Age: int64
Under 30: category
Senior Citizen: category
Married: category
Dependents: category
Number of Dependents: int64
Unnamed: 0_location: int64
Location ID: object
Count_location: int64
Country: category
State: category
City: object
Zip Code: int64
Lat Long: object
Latitude: float64
Longitude: float64
Unnamed: 0: int64
Service ID: object
Count: int64
Quarter: category
Referred a Friend: category
Number of Referrals: int64
Tenure in Months: int64
Offer: category
Phone Service: category
Avg Monthly Long Distance Charges: float64
Multiple Lines: category
Internet Service: category
Internet Type: category
Avg Monthly GB Download: int64
Online Security: category
Online Backup: category
Device Protection Plan: category
Premium Tech Support: category
Streaming TV: category
Streaming Movies: category
Streaming Music: category
Unlimited Data: category
Contract: category
Paperless Billing: category

In [5]:
category_columns = []
int_columns = []
float_columns = []
object_columns = []

for col in df.columns:
    dtype = df[col].dtype
    if dtype == 'object':
        object_columns.append(col)
    elif dtype == 'int64':
        int_columns.append(col)
    elif dtype == 'float64':
        float_columns.append(col)
    else:
        category_columns.append(col)  

print("Integer Columns (int64):")
for col in int_columns:
    print(f" - {col}")

print("\nFloat Columns (float64):")
for col in float_columns:
    print(f" - {col}")

print("\nObject Columns (object):")
for col in object_columns:
    print(f" - {col}")

print("\nCategory Columns (category):")
for col in category_columns:
    print(f" - {col}")


Integer Columns (int64):
 - Unnamed: 0_demographics
 - Count_demographics
 - Age
 - Number of Dependents
 - Unnamed: 0_location
 - Count_location
 - Zip Code
 - Unnamed: 0
 - Count
 - Number of Referrals
 - Tenure in Months
 - Avg Monthly GB Download
 - Total Extra Data Charges
 - Unnamed: 0_status
 - Count_status
 - Satisfaction Score
 - Churn Value
 - Churn Score
 - CLTV
 - Unnamed: 0_customerc
 - LoyaltyID
 - Tenure

Float Columns (float64):
 - Latitude
 - Longitude
 - Avg Monthly Long Distance Charges
 - Monthly Charge
 - Total Charges
 - Total Refunds
 - Total Long Distance Charges
 - Total Revenue
 - Monthly Charges
 - Total Charges_customerc

Object Columns (object):
 - Customer ID
 - Location ID
 - City
 - Lat Long
 - Service ID
 - Status ID

Category Columns (category):
 - Gender
 - Under 30
 - Senior Citizen
 - Married
 - Dependents
 - Country
 - State
 - Quarter
 - Referred a Friend
 - Offer
 - Phone Service
 - Multiple Lines
 - Internet Service
 - Internet Type
 - Online Se

In [6]:
from sklearn.preprocessing import LabelEncoder
yes_no_columns = [col for col in category_columns if df[col].nunique() == 2]
print(yes_no_columns)

remaining_category_columns = [col for col in category_columns if col not in yes_no_columns]
print(remaining_category_columns)

['Gender', 'Under 30', 'Senior Citizen', 'Married', 'Dependents', 'Referred a Friend', 'Phone Service', 'Multiple Lines', 'Internet Service', 'Online Security', 'Online Backup', 'Device Protection Plan', 'Premium Tech Support', 'Streaming TV', 'Streaming Movies', 'Streaming Music', 'Unlimited Data', 'Paperless Billing', 'Churn Label', 'Senior Citizen_customerc', 'Partner', 'Dependents_customerc', 'Phone Service_customerc', 'Paperless Billing_customerc', 'Churn']
['Country', 'State', 'Quarter', 'Offer', 'Internet Type', 'Contract', 'Payment Method', 'Quarter_status', 'Customer Status', 'Churn Category', 'Churn Reason', 'Multiple Lines_customerc', 'Internet Service_customerc', 'Online Security_customerc', 'Online Backup_customerc', 'Device Protection', 'Tech Support', 'Streaming TV_customerc', 'Streaming Movies_customerc', 'Contract_customerc', 'Payment Method_customerc']


In [7]:
# LabelEncoder
le = LabelEncoder()

# Label Encoding
for col in yes_no_columns:
    df[col] = le.fit_transform(df[col])

print(df[yes_no_columns].head())

   Gender  Under 30  Senior Citizen  Married  Dependents  Referred a Friend  \
0       0         0               0        1           0                  1   
1       1         0               0        0           0                  0   
2       1         0               0        0           0                  0   
3       1         0               1        1           0                  1   
4       0         0               1        1           0                  1   

   Phone Service  Multiple Lines  Internet Service  Online Security  ...  \
0              1               0                 1                0  ...   
1              1               1                 1                0  ...   
2              1               0                 1                0  ...   
3              1               0                 1                0  ...   
4              1               0                 1                0  ...   

   Streaming Music  Unlimited Data  Paperless Billing  Churn Label  

In [8]:
# One-hot encoding
df_encoded = pd.get_dummies(df, columns=remaining_category_columns)

print(df_encoded.head())

   Unnamed: 0_demographics Customer ID  Count_demographics  Gender  Age  \
0                     4006  0002-ORFBO                   1       0   37   
1                     4788  0003-MKNFE                   1       1   46   
2                     1901  0004-TLHLJ                   1       1   50   
3                      395  0011-IGKFF                   1       1   78   
4                      368  0013-EXCHZ                   1       0   75   

   Under 30  Senior Citizen  Married  Dependents  Number of Dependents  ...  \
0         0               0        1           0                     0  ...   
1         0               0        0           0                     0  ...   
2         0               0        0           0                     0  ...   
3         0               1        1           0                     0  ...   
4         0               1        1           0                     0  ...   

   Streaming Movies_customerc_No  \
0                              1   
1 